In [297]:
from numpy.random import RandomState
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse

df = pd.read_csv('movies.csv')
#str -> dict
df['類別'] = df['類別'].apply(eval)

# list first catagory
def list_genres(x):
    for d in x.values():
        return(d)

df['類別'] = df['類別'].apply(list_genres)

In [298]:
#split overview
overview = []
for d in df['劇情介紹']:
    keywords = jieba.analyse.extract_tags(d, topK=8)
    overview.append(keywords)

df['關鍵字'] = overview
df['關鍵字'].head()


0              [電影, 這部, 小小, 動畫, 超級, 格魯, 導演, 克里斯]
1             [蝙蝠, 一個, 布魯斯, 韋恩, 並從, 中檢視, 靈魂, 伯派]
2             [奧斯卡, 雷托領, 治療, 實驗, 結果, 瑞德, 比斯, 宇宙]
3    [捍衛戰士, 任務, 1986, 湯姆克魯斯, 獨行俠, 派拉蒙, 金斯基, 珍妮佛]
4              [魔鬼, 星未來, 童星, 保羅, 續集, 他們, 特曼, 系列]
Name: 關鍵字, dtype: object

In [299]:
#unique genres
genreList = []
for genre in df['類別']:
    if genre not in genreList:
        genreList.append(genre)

genreList


['動畫',
 '動作',
 '愛情',
 '戰爭',
 '歷史/傳記',
 '勵志',
 '戲劇',
 '溫馨/家庭',
 '冒險',
 '音樂/歌舞',
 '科幻',
 '奇幻',
 '武俠',
 '劇情',
 '犯罪',
 '恐怖',
 '影展',
 '懸疑/驚悚',
 '喜劇',
 '紀錄片']

In [301]:
import re
df['keywords'] = ''
def bag_words(x):
    return (''.join(re.sub('[^\u4e00-\u9fa5]+', '', x['中文名稱'])) + ' ' + ''.join(x['類別']) + ' ' +  ' '.join(x['關鍵字']))
df['keywords'] = df.apply(bag_words, axis = 1)

df['keywords'].head()
#print(df['keywords'])

0            小小兵2：格魯的崛起 動畫 電影 這部 小小 動畫 超級 格魯 導演 克里斯
1                  蝙蝠俠 動作 蝙蝠 一個 布魯斯 韋恩 並從 中檢視 靈魂 伯派
2                  魔比斯 動作 奧斯卡 雷托領 治療 實驗 結果 瑞德 比斯 宇宙
3    捍衛戰士：獨行俠 動作 捍衛戰士 任務 1986 湯姆克魯斯 獨行俠 派拉蒙 金斯基 珍妮佛
4              魔鬼剋星：未來世 動作 魔鬼 星未來 童星 保羅 續集 他們 特曼 系列
Name: keywords, dtype: object

In [302]:
#split Data to train and test
rng = RandomState()
train = df.sample(frac=0.9173, random_state=rng)
test = df.loc[~df.index.isin(train.index)]

In [312]:
# 將文件中的詞語轉換為詞頻矩陣
from sklearn.neighbors import KNeighborsClassifier
cv = CountVectorizer()
cv_mx = cv.fit_transform(train['keywords'])
cosine_sim = cosine_similarity(cv_mx, cv_mx)
print(cosine_sim)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.09534626 ... 0.         0.         0.        ]
 [0.         0.09534626 1.         ... 0.         0.09534626 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.09534626 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
